In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np

In [2]:
def operate():
    # Capture from video file
    vid = cv2.VideoCapture('hello1.mp4')
    
    while(True):
    
        # Capture the video frame
        ret, frame = vid.read()
        #   Resize the frame
        # frame = cv2.flip(cv2.resize(frame, (800, 600)), 1)
        # frame = cv2.resize(frame, (800, 600))

        # Define model for face detection
        # facedec = cv2.CascadeClassifier('model.xml')
        # Detect faces in each frame
        # faces = facedec.detectMultiScale(frame)
        # for (x,y,w,h) in faces:
        #     # Draw rectangle around the face
        #     cv2.rectangle(frame,(x,y),(x+w,y+h),(197,2,106),2)
        # Display each frame

        if not ret:
            break
        cv2.imshow('frame', frame)
    
        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    # After the loop release the cap object 
    vid.release() 
    # Destroy all the windows opened by imshow
    cv2.destroyAllWindows()

In [4]:
operate()